## Selecting coordinate based on latitude and longitude

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import datetime
import scipy.io

%matplotlib inline

Load tidal speed matrix:

In [3]:
mat         = scipy.io.loadmat('/ocean/brogalla/GEOTRACES/data/mean_tide_speed.mat')
tidal_speed = mat['mean_speed']

ts_mesh     = nc.Dataset('/ocean/brogalla/GEOTRACES/data/hsquare.nc')
ts_lons     = np.array(ts_mesh.variables['nav_lon'])
ts_lats     = np.array(ts_mesh.variables['nav_lat'])

Functions:

In [32]:
def find_closest_model_point(lon, lat, model_lons, model_lats, grid='NEMO',
                                tols={'NEMO': {'tol_lon': 0.104, 'tol_lat': 0.0388},
                                    'GEM2.5': {'tol_lon': 0.016, 'tol_lat': 0.012}}):

    # Search for a grid point with longitude and latitude within
    # tolerance of measured location
    j_list, i_list = np.where(
        np.logical_and(
            (np.logical_and(model_lons > lon - tols[grid]['tol_lon'],
                            model_lons < lon + tols[grid]['tol_lon'])),
            (np.logical_and(model_lats > lat - tols[grid]['tol_lat'],
                            model_lats < lat + tols[grid]['tol_lat']))
        )
    )

    if len(j_list) == 0:
        # If including points outside of domain:
        return np.nan, np.nan
    try:
        j, i = map(np.asscalar, (j_list, i_list))
    except ValueError:
        # Several points within tolerance
        # Calculate distances for all and choose the closest
        lons = [model_lons[j_list[n], i_list[n]] for n in range(len(j_list))]
        lats = [model_lats[j_list[n], i_list[n]] for n in range(len(j_list))]
        dists = haversine(
            np.array([lon] * i_list.size), np.array([lat] * j_list.size),
            lons, lats)
        n = dists.argmin()
        j, i = map(np.asscalar, (j_list[n], i_list[n]))

    return j, i

def haversine(lon1, lat1, lon2, lat2):
    """Calculate the great-circle distance in kilometers between two points
    on a sphere from their longitudes and latitudes.

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

#### Example: 

Find nearest model point to let's say (72.00N, -67.00W)

In [33]:
stn_lon , stn_lat = -67.00, 72.00

Call functions:

In [34]:
stn_indx , stn_indy  = find_closest_model_point(stn_lon, stn_lat, ts_lons, ts_lats)

In [35]:
print(f'(x, y) index of station in tidal speed matrix: ({stn_indx}, {stn_indy})')

(x, y) index of station in tidal speed matrix: (nan, nan)


Gives "nan" if the coordinate is not within the specified tolerance. So, let's increase the tolerance of closeness for the latitude and longitude:

In [36]:
stn_indx , stn_indy  = find_closest_model_point(stn_lon, stn_lat, ts_lons, ts_lats, \
                                                tols={'NEMO': {'tol_lon': 0.1, 'tol_lat': 0.1}})

/home/brogalla/miniconda3/envs/julab/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


In [37]:
print(f'(x, y) index of station: ({stn_indx}, {stn_indy})')

(x, y) index of station: (486, 187)


So, we have found the coordinate of the specified station latitude and longitude that falls within 0.1 degrees latitude and longitude of the tidal speed grid.

In [42]:
print(f'Location that we were searching for: ({stn_lon},{stn_lat})')
print(f'Closest location that we found in array: ({ts_lons[stn_indx,stn_indy]}, {ts_lats[stn_indx,stn_indy]})')
print(f'The tidal speed associated with this coordinate is: {tidal_speed[stn_indx, stn_indy]}')

Location that we were searching for: (-67.0,72.0)
Closest location that we found in array: (-66.93526458740234, 71.91983795166016)
The tidal speed associated with this coordinate is: 0.03341489260568193
